In [2]:
import torch
from torch import nn
from torchvision import models
from torch.utils.data import DataLoader
import numpy as np
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, ToPILImage
from PIL import Image
import cv2
import pandas as pd
from pathlib import Path
import os
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.cuda.is_available())
torch.cuda.empty_cache()

cuda
True


In [3]:
train_images = "./DATA/trainImages"
val_images = "./DATA/valImages"
test_images = "./DATA/testImages"

train_roi_jsonl = "./DATA/data.jsonl"
val_roi_jsonl = "./DATA/val.jsonl"
test_roi_jsonl = "./DATA/test.jsonl"


train_csv = "./DATA/train.csv"
val_csv = "./DATA/val.csv"
test_csv = "./DATA/test.csv"

In [ ]:
import cv2
from ultralytics import YOLO
import os
import json

model = YOLO('yolov8x.pt')  

data = []
images_path = './DATA/testImages'

ls = os.listdir(images_path)
ls.sort(key=lambda x: int(x.split(".")[0]))
for i, image_path in enumerate(ls):
    # Read the image
    image = cv2.imread(os.path.join(images_path, image_path))
    results = model(image)
    res = []
    # Extract bounding boxes, labels, and confidence scores
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()  # Extract bounding box coordinates
        scores = result.boxes.conf.cpu().numpy()  # Extract confidence scores
        labels = result.boxes.cls.cpu().numpy()  # Extract class labels
        # Draw bounding boxes on the image
        for box, score, label in zip(boxes, scores, labels):
            x1, y1, x2, y2 = map(int, box)
            class_name = model.names[int(label)]
            # cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label_text = f'{class_name} {score:.2f}'
            # cv2.putText(image, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            res.append({
                "label": label_text,
                "class_name": class_name,
                "box": [[x1, y1], [x2, y2]]
            })
    data.append({
        "id": image_path.split("/")[-1],
        "image": image_path,
        "boxes": res
    })
    
with open("test.jsonl", 'w') as f:
    for obj in data:
        f.write(json.dumps(obj, ensure_ascii=False) + '\n')

In [5]:
from transformers import CLIPProcessor, CLIPModel, RobertaModel, RobertaTokenizer
import torch

clip_model = torch.jit.load("./models/clip_model.pt").cuda().eval()
input_resolution = clip_model.input_resolution.item()
context_length = clip_model.context_length.item()
vocab_size = clip_model.vocab_size.item()

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in clip_model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

preprocess = Compose([
    Resize(input_resolution, interpolation=Image.BICUBIC),
    CenterCrop(input_resolution),
    ToTensor()
    ])

# Load RoBERTa model and tokenizer
roberta_model = RobertaModel.from_pretrained('roberta-base').cuda().eval()
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Preprocess image for CLIP
def process_image_clip(in_img):
    image_mean = torch.tensor([0.48145466, 0.4578275, 0.40821073])
    image_std = torch.tensor([0.26862954, 0.26130258, 0.27577711])
    
    image = preprocess(Image.open(in_img).convert("RGB"))
    
    image_input = torch.tensor(np.stack(image))
    image_input -= image_mean[:, None, None]
    image_input /= image_std[:, None, None]
    return image_input


# Preprocess text using RoBERTa
def process_text_roberta(in_text, max_length=77):
    # tokens = roberta_tokenizer(in_text, return_tensors="pt", truncation=True, padding="max_length", max_length=75)
    # tokens = {k: v.to("cuda") for k, v in tokens.items()}
    # text_embedding = roberta_model(**tokens).last_hidden_state
    # return text_embedding
    return roberta_tokenizer(
                    text=in_text, 
                    return_tensors="pt", 
                    padding='max_length', 
                    max_length=max_length, 
                    truncation=True
                ).to(device)



Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/miniconda/envs/CAPSTONE/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
class FeatureExtractor(nn.Module):
    def __init__(self, model):
        super(FeatureExtractor, self).__init__()
        # Extract VGG-16 Feature Layers
        self.features = list(model.features)
        self.features = nn.Sequential(*self.features)
        # Extract VGG-16 Average Pooling Layer
        self.pooling = model.avgpool
        # Convert the image into one-dimensional vector
        self.flatten = nn.Flatten()
        # Extract the first part of fully-connected layer from VGG16
        self.fc = model.classifier[0]
  
    def forward(self, x):
        # It will take the input 'x' until it returns the feature vector called 'out'
        out = self.features(x)
        out = self.pooling(out)
        out = self.flatten(out)
        out = self.fc(out) 
        return out 

# Initialize the model
model_vgg_pretrained = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
model_vgg = FeatureExtractor(model_vgg_pretrained)

# Change the device to GPU
model_vgg = model_vgg.to(device)

# Transform the image, so it becomes readable with the model
transform_vgg_BB = Compose([
  ToPILImage(),
  CenterCrop(512),
  Resize((448,448)),
  ToTensor()                              
])


# Iterate each image
def get_image_vgg_BB(l, t, r, b, in_im): 
#     left, top, right, bottom and input image
    img = cv2.imread(in_im)
    h, w, _ = img.shape
    # crop
    x1 = int(np.floor(l*w))
    x2 = int(np.floor(r*w))
    y1 = int(np.floor(b*h))
    y2 = int(np.floor(t*h))
    crop_img = img[y1:y2, x1:x2]    
    
    # Transform the cropped image
    img = transform_vgg_BB(crop_img)
    # Reshape the image. PyTorch model reads 4-dimensional tensor
    # [batch_size, channels, width, height]
    img = img.reshape(1, 3, 448, 448)
    img = img.to(device)
    # We only extract features, so we don't need gradient
    with torch.no_grad():
        # Extract the feature from the image
        feature = model_vgg(img).squeeze()
    # Convert to NumPy Array, Reshape it, and save it to features variable
    return feature

# Transform the image, so it becomes readable with the model
transform_vgg_center = Compose([
  ToPILImage(),
  CenterCrop(512),
  Resize(448),
  ToTensor()                              
])

# Iterate each image
def get_image_vgg_center(in_im):
    # Set the image path
    path = in_im
    # Read the file
    img = cv2.imread(path)
    # Transform the image
    img = transform_vgg_center(img)
    # Reshape the image. PyTorch model reads 4-dimensional tensor
    # [batch_size, channels, width, height]
    img = img.reshape(1, 3, 448, 448)
    img = img.to(device)
    # We only extract features, so we don't need gradient
    with torch.no_grad():
        # Extract the feature from the image
        feature = model_vgg(img).squeeze()
    
    return feature



In [21]:
from sentence_transformers import SentenceTransformer

model_sent_trans = SentenceTransformer('sentence-transformers/all-mpnet-base-v2').cuda().eval()

class DatasetCreater(torch.utils.data.Dataset):
    """Dataset class to preprocess and serve multimodal tensors for model input."""

    def __init__(
        self,
        data_path,
        img_dir,
        data_path_roi=None,
        split_flag=None,
        balance=False,
        dev_limit=None,
        random_state=0,
    ):
        self.split_flag = split_flag
        
        delimeter = '\t' if split_flag == "val" else ","
        
        self.samples_frame = pd.read_csv(data_path, delimiter=delimeter)
        
        if data_path_roi is not None:
            self.roi_frame = pd.read_json(data_path_roi, lines=True)
            self.roi_frame.image = self.roi_frame.apply(lambda row: (img_dir + '/' + row["image"]), axis=1)
            
        self.samples_frame = self.samples_frame.reset_index(drop=True)

    def __len__(self):
        return len(self.samples_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_id = self.roi_frame.loc[idx, "id"]
        img_file_name = self.roi_frame.loc[idx, "image"]

        # Process image using CLIP's image processor
        image_clip_input = process_image_clip(self.roi_frame.loc[idx, "image"])

        # On-demand VGG feature extraction or pre-extracted
        BB_info = self.roi_frame.loc[idx, "boxes"]
        roi_vgg_feat_list = []
        if len(BB_info):
            total_BB = len(BB_info)
            BB_info_final = BB_info[:4] if total_BB > 4 else BB_info
            for item in BB_info_final:
                left = item['box'][0][0]
                top = item['box'][0][1]
                right = item['box'][1][0]
                bottom = item['box'][1][1]
                roi_vgg_feat = get_image_vgg_BB(left, top, right, bottom, img_file_name)
                roi_vgg_feat_list.append(roi_vgg_feat)
            image_vgg_feature = torch.mean(torch.vstack(roi_vgg_feat_list), axis=0).to('cuda')
        else:
            image_vgg_feature = get_image_vgg_center(img_file_name).clone().detach().to('cuda')

        # Process text using RoBERTa
        text_roberta_input = process_text_roberta(self.samples_frame.loc[idx, "ocr"])

        # Process entities (entity embeddings with sentence-transformers)
        cur_ent_list = [item["class_name"] for item in self.roi_frame.loc[idx, "boxes"]]
        cur_ent_rep_list = [torch.tensor(model_sent_trans.encode(item)).to(device) for item in cur_ent_list]
        text_drob_feature = torch.mean(torch.vstack(cur_ent_rep_list), axis=0).to(device) if len(cur_ent_list) else torch.tensor(model_sent_trans.encode(self.samples_frame.loc[idx, "ocr"])).to(device)

        if self.split_flag == 'test':
            sample = {
                "id": img_id,
                "image_clip_input": image_clip_input.to("cuda"),
                "image_vgg_feature": image_vgg_feature,
                "text_roberta_input": text_roberta_input.to("cuda"),
                "text_drob_embedding": text_drob_feature,
            }
        else:    
            lab = 0 if self.samples_frame.loc[idx, "offensive"] == "not_offensive" else 1
            label = torch.tensor(lab).to('cuda')
            sample = {
                "id": img_id,
                "image_clip_input": image_clip_input.to("cuda"),
                "image_vgg_feature": image_vgg_feature,
                "text_roberta_input": text_roberta_input.to("cuda"),
                "text_drob_embedding": text_drob_feature,
                "label": label
            }

        return sample



/opt/miniconda/envs/CAPSTONE/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [22]:
torch.cuda.empty_cache()

dataset_train = DatasetCreater(train_csv, train_images, data_path_roi=train_roi_jsonl, split_flag='train')
dataloader_train = DataLoader(dataset_train, batch_size=32,
                        shuffle=True, num_workers=0)
dataset_val = DatasetCreater(val_csv, val_images, data_path_roi=val_roi_jsonl ,split_flag='val')
dataloader_val = DataLoader(dataset_val, batch_size=32,
                        shuffle=True, num_workers=0)
dataset_test = DatasetCreater(test_csv, test_images, data_path_roi=test_roi_jsonl, split_flag='test')
dataloader_test = DataLoader(dataset_test, batch_size=32,
                        shuffle=False, num_workers=0)


In [13]:
import torch.nn as nn
import torch.nn.functional as F 

class Momenta(nn.Module):
    def __init__(self, n_out):
        super(Momenta, self).__init__()  
        
        # Dense layers for VGG and RoBERTa features
        self.dense_vgg_1024 = nn.Linear(4096, 1024)
        self.dense_vgg_512 = nn.Linear(1024, 512)
        self.drop20 = nn.Dropout(p=0.2)
        self.drop5 = nn.Dropout(p=0.05) 
        
        self.dense_drob_512 = nn.Linear(768, 512)
        
        # Self-attention layers
        self.gen_key_L1 = nn.Linear(512, 256) 
        self.gen_query_L1 = nn.Linear(512, 256)
        self.gen_middle_L2 = nn.Linear(768, 512)
        self.gen_key_L2 = nn.Linear(512, 256)
        self.gen_query_L2 = nn.Linear(512, 256) 
        self.gen_key_L3 = nn.Linear(512, 256)
        self.gen_query_L3 = nn.Linear(512, 256)

        self.soft = nn.Softmax(dim=1)
        self.project_dense_512a = nn.Linear(1024, 512)
        self.project_dense_512b = nn.Linear(1024, 512)
        self.project_dense_512c = nn.Linear(1024, 512)
        
        # Final layers
        self.fc_out = nn.Linear(512, 256)
        self.out = nn.Linear(256, n_out)
        

    def selfattNFuse_L1a(self, vec1, vec2): 
        q1 = F.relu(self.gen_query_L1(vec1))
        k1 = F.relu(self.gen_key_L1(vec1))
        q2 = F.relu(self.gen_query_L1(vec2))
        k2 = F.relu(self.gen_key_L1(vec2))
        score1 = torch.reshape(torch.bmm(q1.view(-1, 1, 256), k2.view(-1, 256, 1)), (-1, 1))
        score2 = torch.reshape(torch.bmm(q2.view(-1, 1, 256), k1.view(-1, 256, 1)), (-1, 1))
        wt_score1_score2_mat = torch.cat((score1, score2), 1)
        wt_i1_i2 = self.soft(wt_score1_score2_mat.float())
        prob_1 = wt_i1_i2[:, 0]
        prob_2 = wt_i1_i2[:, 1]
        wtd_i1 = vec1 * prob_1[:, None]
        wtd_i2 = vec2 * prob_2[:, None]
        out_rep = F.relu(self.project_dense_512a(torch.cat((wtd_i1, wtd_i2), 1)))
        return out_rep

    def selfattNFuse_L1b(self, vec1, vec2):
        vec2_pool = torch.mean(vec2, dim=1).to(device)
        q1 = F.relu(self.gen_query_L2(vec1))
        k1 = F.relu(self.gen_key_L2(vec1))
        q2a = self.gen_middle_L2(vec2_pool)
        q2 = F.relu(self.gen_query_L2(q2a))
        k2 = F.relu(self.gen_key_L2(q2a))
        score1 = torch.reshape(torch.bmm(q1.view(-1, 1, 256), k2.view(-1, 256, 1)), (-1, 1))
        score2 = torch.reshape(torch.bmm(q2.view(-1, 1, 256), k1.view(-1, 256, 1)), (-1, 1))
        wt_score1_score2_mat = torch.cat((score1, score2), 1)
        wt_i1_i2 = self.soft(wt_score1_score2_mat.float())
        prob_1 = wt_i1_i2[:, 0]
        prob_2 = wt_i1_i2[:, 1]
        wtd_i1 = vec1 * prob_1[:, None]
        wtd_i2 = q2a * prob_2[:, None]
        out_rep = F.relu(self.project_dense_512b(torch.cat((wtd_i1, wtd_i2), 1)))
        return out_rep
    
    def selfattNFuse_L2(self, vec1, vec2): 
        q1 = F.relu(self.gen_query_L3(vec1))
        k1 = F.relu(self.gen_key_L3(vec1))
        q2 = F.relu(self.gen_query_L3(vec2))
        k2 = F.relu(self.gen_key_L3(vec2))
        score1 = torch.reshape(torch.bmm(q1.view(-1, 1, 256), k2.view(-1, 256, 1)), (-1, 1))
        score2 = torch.reshape(torch.bmm(q2.view(-1, 1, 256), k1.view(-1, 256, 1)), (-1, 1))
        wt_score1_score2_mat = torch.cat((score1, score2), 1)
        wt_i1_i2 = self.soft(wt_score1_score2_mat.float())
        prob_1 = wt_i1_i2[:, 0]
        prob_2 = wt_i1_i2[:, 1]
        wtd_i1 = vec1 * prob_1[:, None]
        wtd_i2 = vec2 * prob_2[:, None]
        out_rep = F.relu(self.project_dense_512c(torch.cat((wtd_i1, wtd_i2), 1)))
        return out_rep

    def forward(self, in_CI, in_VGG, in_CT, in_Drob):        
        VGG_feat = self.drop20(F.relu(self.dense_vgg_512(self.drop20(F.relu(self.dense_vgg_1024(in_VGG))))))
        Drob_feat = self.drop5(F.relu(self.dense_drob_512(in_Drob)))
        out_img = self.selfattNFuse_L1a(VGG_feat, in_CI)
        out_txt = self.selfattNFuse_L1b(Drob_feat, in_CT)        
        out_img_txt = self.selfattNFuse_L2(out_img, out_txt)
        final_out = F.relu(self.fc_out(out_img_txt))
        out = self.out(final_out)
        return out

In [14]:
from tqdm import tqdm
from early_stopping_pytorch.pytorchtools import EarlyStopping

def train_model(model, patience, n_epochs, dataloader_train, dataloader_val, clip_model, optimizer, criterion, exp_path, device):
    epochs = n_epochs
    train_acc_list, val_acc_list, train_loss_list, val_loss_list = [], [], [], []
    
    # Path(exp_path).mkdir(parents=True, exist_ok=True)
    chk_file = os.path.join(exp_path, 'checkpoint.pt')
    early_stopping = EarlyStopping(patience=patience, verbose=True, path=chk_file)
    
    for epoch in range(epochs):
        model.cuda().train()
        total_loss_train, total_train, correct_train = 0, 0, 0

        for data in tqdm(dataloader_train):
            img_inp_clip = data['image_clip_input'].to(device)
            txt_inp_clip = data['text_roberta_input'].to(device)
            
            input_ids = txt_inp_clip['input_ids'].squeeze(1).to(device)
            attention_mask = txt_inp_clip['attention_mask'].squeeze(1).to(device)
            # img_feat_clip = clip_model(pixel_values=img_inp_clip).image_embeds
            # txt_feat_clip = clip_model(input_ids=txt_inp_clip).text_embeds
            
            with torch.no_grad():
                img_feat_clip = clip_model.encode_image(img_inp_clip).float().to(device)
                txt_feat_clip = roberta_model(
                            input_ids=input_ids,
                            attention_mask=attention_mask
                ).last_hidden_state.to(device)

                # img_feat_clip = clip_model(img_inp_clip).float().to(device)
                # txt_feat_clip = clip_model(txt_inp_clip).float().to(device)
            
            
            img_feat_vgg = data['image_vgg_feature'].to(device)
            txt_feat_trans = data['text_drob_embedding'].to(device)
            label_train = data['label'].to(device)

            model.zero_grad()
            output = model(img_feat_clip, img_feat_vgg, txt_feat_clip, txt_feat_trans)
            loss = criterion(output, label_train)
            
            loss.backward()
            optimizer.step()

            # print(".", end= ')
            
            _, predicted_train = torch.max(output, 1)
            total_train += label_train.size(0)
            correct_train += (predicted_train == label_train).sum().item()
            total_loss_train += loss.item()

        train_acc = 100 * correct_train / total_train
        train_loss = total_loss_train / total_train
        train_acc_list.append(train_acc)
        train_loss_list.append(train_loss)

        model.cuda().eval()
        total_loss_val, total_val, correct_val = 0, 0, 0
        with torch.no_grad():
            for data in tqdm(dataloader_val):
                img_inp_clip = data['image_clip_input'].to(device)
                txt_inp_clip = data['text_roberta_input'].to(device)
                
                input_ids = txt_inp_clip['input_ids'].squeeze(1).to(device)
                attention_mask = txt_inp_clip['attention_mask'].squeeze(1).to(device)
                # img_feat_clip = clip_model(pixel_values=img_inp_clip).image_embeds
                # txt_feat_clip = clip_model(input_ids=txt_inp_clip).text_embeds
                
                with torch.no_grad():
                    img_feat_clip = clip_model.encode_image(img_inp_clip).float().to(device)
                    txt_feat_clip = roberta_model(
                                input_ids=input_ids,
                                attention_mask=attention_mask
                    ).last_hidden_state.to(device)

                # img_feat_clip = clip_model(img_inp_clip).float().to(device)
                # txt_feat_clip = clip_model(txt_inp_clip).float().to(device)
            
                img_feat_vgg = data['image_vgg_feature'].to(device)
                txt_feat_trans = data['text_drob_embedding'].to(device)
                label_val = data['label'].to(device)

                output = model(img_feat_clip, img_feat_vgg, txt_feat_clip, txt_feat_trans)
                val_loss = criterion(output, label_val)

                _, predicted_val = torch.max(output, 1)
                total_val += label_val.size(0)
                correct_val += (predicted_val == label_val).sum().item()
                total_loss_val += val_loss.item()
                
        print("\nSaving model...") 
        torch.save(model.state_dict(), os.path.join(exp_path, "momenta_1.pt"))

        val_acc = 100 * correct_val / total_val
        val_loss = total_loss_val / total_val
        val_acc_list.append(val_acc)
        val_loss_list.append(val_loss)

        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break

        print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f}, train_acc: {train_acc:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}')
        
        # model.train()

    return model, train_acc_list, val_acc_list, train_loss_list, val_loss_list

In [ ]:
output_size = 2
patience = 10
n_epochs = 25
lr = 0.001
criterion = nn.CrossEntropyLoss()
model_exp_path = "models/"


model = Momenta(output_size)

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)

model, train_acc_list, val_acc_list, train_loss_list, val_lost_lost = train_model(
                                                                model, patience, n_epochs, 
                                                                dataloader_train, dataloader_val, 
                                                                clip_model, optimizer, criterion, model_exp_path, device)



In [31]:
model.load_state_dict(torch.load("./models/momenta_1.pt"))


data = iter(dataloader_test)
data = next(data)
# print(data)
img_inp_clip = data['image_clip_input'].to(device)
txt_inp_clip = data['text_roberta_input'].to(device)

input_ids = txt_inp_clip['input_ids'].squeeze(1).to(device)
attention_mask = txt_inp_clip['attention_mask'].squeeze(1).to(device)
# img_feat_clip = clip_model(pixel_values=img_inp_clip).image_embeds
# txt_feat_clip = clip_model(input_ids=txt_inp_clip).text_embeds

model.cuda().eval()
with torch.no_grad():
    img_feat_clip = clip_model.encode_image(img_inp_clip).float().to(device)
    txt_feat_clip = roberta_model(
                input_ids=input_ids,
                attention_mask=attention_mask
    ).last_hidden_state.to(device)

# img_feat_clip = clip_model(img_inp_clip).float().to(device)
# txt_feat_clip = clip_model(txt_inp_clip).float().to(device)

img_feat_vgg = data['image_vgg_feature'].to(device)
txt_feat_trans = data['text_drob_embedding'].to(device)

output = model(img_feat_clip, img_feat_vgg, txt_feat_clip, txt_feat_trans)
label = torch.tensor(1).to('cuda')
_, predicted_val = torch.max(output, 1)
correct_val = (predicted_val == label).sum().item()
print(predicted_val)
print(correct_val)
print(output.shape)


tensor([1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], device='cuda:0')
6
torch.Size([32, 3])


In [32]:
output

tensor([[-5.9377e-02,  6.8642e-01, -1.4575e+01],
        [ 5.4813e-01, -2.5671e-01, -8.3601e+00],
        [ 3.9612e-01, -1.6611e-01, -7.4256e+00],
        [ 1.0893e-01,  1.6286e-01, -9.3442e+00],
        [ 7.6036e-01, -4.6514e-01, -1.0472e+01],
        [ 1.9329e-01,  3.4533e-03, -6.5389e+00],
        [ 6.6558e-01, -3.8987e-01, -8.8520e+00],
        [ 2.1970e-01, -1.9197e-03, -6.6199e+00],
        [ 1.6921e+00, -5.9901e-01, -4.3492e+01],
        [ 1.4489e+00, -7.2455e-01, -2.4093e+01],
        [ 1.4234e-01,  2.8512e-01, -1.4927e+01],
        [ 3.8332e-01, -1.5771e-01, -7.3385e+00],
        [ 1.9191e+00, -1.0038e+00, -3.1869e+01],
        [-7.2446e-01,  1.7029e+00, -3.7451e+01],
        [ 5.0031e-01, -2.2225e-01, -8.0954e+00],
        [ 6.1174e-01, -3.2348e-01, -8.5911e+00],
        [ 1.9152e+00, -9.9996e-01, -3.0747e+01],
        [ 6.8701e-01, -4.2694e-01, -8.3729e+00],
        [ 4.3143e-01, -1.6852e-01, -7.8351e+00],
        [ 6.0749e-01, -3.0195e-01, -8.7815e+00],
        [ 7.0547e-01